In [2]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(magrittr); library(zeallot); library(glue); library(tidyverse); library(glue); library(lubridate)
    library(scales);library(reshape2);library(RColorBrewer);
})

In [3]:
## Run pageview query and load data
query <- "
SELECT
date, apps, desktop, mobileweb, total
FROM cchen.pageviews_corrected
WHERE year >= 2014
"

In [4]:
pageviews_all <- wmf::query_hive(query)

In [8]:
query <- "SELECT 
    CONCAT(year,'-',LPAD(month,2,'0'),'-',LPAD(day,2,'0')) AS date,
    SUM(IF(access_method = 'mobile app', view_count, null)) AS apps,
    SUM(IF(access_method = 'desktop', view_count, null)) AS desktop,
    SUM(IF(access_method = 'mobile web', view_count, null)) AS mobileweb,
    SUM(view_count) as total,
    year, month, day
FROM wmf.pageview_hourly
WHERE (year = 2020 AND month = 3)
    AND agent_type='user'
    AND NOT (country_code IN ('PK', 'IR', 'AF') -- https://phabricator.wikimedia.org/T157404#3194046
    AND user_agent_map['browser_family'] = 'IE') -- https://phabricator.wikimedia.org/T193578#4300284
GROUP BY year, month, day"

In [9]:
pageviews_mar <- wmf::query_hive(query)

In [12]:
pageviews_mar <- pageviews_mar %>%
    select (date, apps, desktop, mobileweb, total)

In [13]:
pageviews_all <- rbind(pageviews_all, pageviews_mar)

In [22]:
pageviews_all$date <- as.Date(pageviews_all$date, format = "%Y-%m-%d")

In [35]:
pageviews_all %>% select(date,total) %>% arrange(desc(total))

date,total
<date>,<int>
2020-03-22,605694491
2019-11-12,603469671
2020-03-24,599509244
2020-01-27,598292019
2019-11-13,597683456
2020-02-10,596006383
2017-02-27,595389171
2017-02-13,594548277
2016-11-09,594233977
